In [1]:
import codecs

positive = [] 
negative = [] 
posneg = [] 
pos = codecs.open("./data/positive_words_self.txt", 'rb', encoding='UTF-8') 

while True: 
    line = pos.readline() 
    line = line.replace('\n', '')
    
    if not line: break
        
    positive.append(line) 
    posneg.append(line) 
    
     
        
pos.close() 

neg = codecs.open("./data/negative_words_self.txt", 'rb', encoding='UTF-8') 
while True: 
    line = neg.readline() 
    line = line.replace('\n', '')
    if not line: break
    
    negative.append(line) 
    posneg.append(line) 
    
neg.close()


In [2]:
import pandas as pd
import numpy as np
import pymysql
import re

In [22]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}
media_list = {'아시아경제_' : 'asia_',
             '매일경제_': 'maeil_'}

df_list = []

for n,m in media_list.items():
    for i,j in com_list.items():

        db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

        cursor = db.cursor(pymysql.cursors.DictCursor)
        sql = "select * from "+ m +"news_craw_" + j +" where length(date) = 10"
        
        cursor.execute(sql)

        result = cursor.fetchall()

        # 데이터 프레임으로 변경
        globals() ['df_'+ m + j ] = pd.DataFrame(result)
        print('df_'+ m + j)
        df_list.append(globals() ['df_'+ m + j ])
        
        db.close()

df_asia_000660
df_asia_005380
df_asia_005930
df_asia_051910
df_asia_068270
df_maeil_000660
df_maeil_005380
df_maeil_005930
df_maeil_051910
df_maeil_068270


In [26]:
df_total = pd.concat(df_list)

In [27]:
df_total

,st_n,st_cd,news,date,title,url,text
0,sk하이닉스,000660,아시아경제,2018122715,"SK하이닉스, 우수 엔지니어 60세 넘어도 일할 수 있다",http://view.asiae.co.kr/news/view.htm?idxno=20...,2020년에는 상대평가제도도 폐지…소통강화 위해 호칭도 'TL'로 통일 [아시아경제...
1,sk하이닉스,000660,아시아경제,2018123110,"오션브릿지, SK하이닉스와 12억 규모 반도체 장비제조 공급계약 체결",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 오주연 기자] \r\t\t\t\t\t\t\...
2,sk하이닉스,000660,아시아경제,2018122816,"SK하이닉스, 우수 엔지니어 정년 없앤다",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t인사제도 파격적 개편…최태원 회장의 딥체인지 실현 [아...
3,sk하이닉스,000660,아시아경제,2018122607,"[클릭 e종목]""SK하이닉스, 내년 반도체 수요 증가율 상저하고…목표가↓""",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 송화정 기자]신한금융투자는 26일 \r\t...
4,sk하이닉스,000660,아시아경제,2018122707,"[클릭 e종목]""SK하이닉스, 4Q영업익 전망치 하향""",http://view.asiae.co.kr/news/view.htm?idxno=20...,\r\t\t\t\t\t\t\t[아시아경제 오주연 기자] 하나금융투자가 27일 \r\...
...,...,...,...,...,...,...,...
3216,셀트리온,068270,매일경제,2021092309,"""미국 중국 동시 악재라니""…동학개미 악소리 난다, 연휴 뒤 코스피 급락",http://news.mk.co.kr/newsRead.php?no=907885&ye...,\n\n\n[사진 출처 = 연합뉴스]\n 중국 헝다그룹 유동성 위기 등 추석 연휴에...
3217,셀트리온,068270,매일경제,2021092317,"디지털 강자 매경, 카카오 블록체인 클레이튼과 손잡다",http://news.mk.co.kr/newsRead.php?no=910365&ye...,\n\n\n올해로 22회째를 맞은 세계지식포럼에서 로버트 주브린 화성학회 회장이 2...
3218,셀트리온,068270,매일경제,2021092715,"코스피 외국인 기관 쌍끌이 매수…8.40포인트↑, 3133.64 장 마감",http://news.mk.co.kr/newsRead.php?no=919526&ye...,\n\n\n[사진 출처 = 연합뉴스]\n 중국 부동산 재벌 헝다그룹의 파산 우려에도...
3219,셀트리온,068270,매일경제,2021092414,"휴마시스, 셀트리온과 273억원 규모 코로나19 항원진단키트 공급 계약",http://news.mk.co.kr/newsRead.php?no=912835&ye...,현장형 신속검사 기업 휴마시스가 셀트리온과 273억원 규모 코로나19 항원진단키트 ...


In [13]:
j = 0
label = [0] * len(df_total)

my_dic = {"text":[], "label":label}

texts = df_total.text
texts = texts.str.replace('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“▲》◆▶◇■]','')
texts = texts.str.replace('[\n\r\t\\\]','')
texts = texts.reset_index().drop('index',1).text.str[:1000]

for df in range(len(df_total)):    
    txt_data = texts[df]
    
    my_dic['text'].append(txt_data) 

    for i in range(len(posneg)): 
        posflag = False 
        negflag = False 

        if i < (len(positive)-1): 
            # print(title_data.find(posneg[i])) 
            if txt_data.find(posneg[i]) != -1: 
                posflag = True 

                print(i, "positive?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break 

        if i > (len(positive)-2): 
            if txt_data.find(posneg[i]) != -1: 
                negflag = True 
                print(i, "negative?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break 

    if posflag == True: 
        label[j] = 1 
        # print("positive", j) 

    elif negflag == True: 
        label[j] = -1 
        # print("negative", j) 

    elif negflag == False and posflag == False: 
        label[j] = 0 
        # print("objective", j) 

    j = j + 1 
                
my_dic['label'] = label 
my_df = pd.DataFrame(my_dic)


37 positive? 테스트 :  96 비교단어 :  새로운 인덱스 :  37
29 positive? 테스트 :  211 비교단어 :  체결 인덱스 :  29
30 positive? 테스트 :  203 비교단어 :  증가 인덱스 :  30
62 negative? 테스트 :  435 비교단어 :  산재 인덱스 :  62
37 positive? 테스트 :  434 비교단어 :  새로운 인덱스 :  37
13 positive? 테스트 :  159 비교단어 :  유치 인덱스 :  13
15 positive? 테스트 :  843 비교단어 :  출시 인덱스 :  15
32 positive? 테스트 :  428 비교단어 :  기부 인덱스 :  32
32 positive? 테스트 :  437 비교단어 :  기부 인덱스 :  32
95 negative? 테스트 :  30 비교단어 :  우려 인덱스 :  95
15 positive? 테스트 :  820 비교단어 :  출시 인덱스 :  15
29 positive? 테스트 :  212 비교단어 :  체결 인덱스 :  29
30 positive? 테스트 :  902 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  456 비교단어 :  하락 인덱스 :  59
79 negative? 테스트 :  232 비교단어 :  문제 인덱스 :  79
36 positive? 테스트 :  189 비교단어 :  최고 인덱스 :  36
6 positive? 테스트 :  204 비교단어 :  수상 인덱스 :  6
30 positive? 테스트 :  153 비교단어 :  증가 인덱스 :  30
29 positive? 테스트 :  202 비교단어 :  체결 인덱스 :  29
29 positive? 테스트 :  232 비교단어 :  체결 인덱스 :  29
14 positive? 테스트 :  18 비교단어 :  1위 인덱스 :  14
30 positive? 테스트 :  410 비교단어 :  증가 인덱스 :  30
52 negative? 

88 negative? 테스트 :  410 비교단어 :  게이트 인덱스 :  88
37 positive? 테스트 :  475 비교단어 :  새로운 인덱스 :  37
59 negative? 테스트 :  639 비교단어 :  하락 인덱스 :  59
106 negative? 테스트 :  95 비교단어 :  소송 인덱스 :  106
34 positive? 테스트 :  484 비교단어 :  신제품 인덱스 :  34
29 positive? 테스트 :  360 비교단어 :  체결 인덱스 :  29
29 positive? 테스트 :  219 비교단어 :  체결 인덱스 :  29
30 positive? 테스트 :  25 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  288 비교단어 :  하락 인덱스 :  59
14 positive? 테스트 :  422 비교단어 :  1위 인덱스 :  14
29 positive? 테스트 :  364 비교단어 :  체결 인덱스 :  29
17 positive? 테스트 :  139 비교단어 :  오픈 인덱스 :  17
30 positive? 테스트 :  222 비교단어 :  증가 인덱스 :  30
29 positive? 테스트 :  116 비교단어 :  체결 인덱스 :  29
27 positive? 테스트 :  662 비교단어 :  진출 인덱스 :  27
37 positive? 테스트 :  615 비교단어 :  새로운 인덱스 :  37
29 positive? 테스트 :  133 비교단어 :  체결 인덱스 :  29
2 positive? 테스트 :  883 비교단어 :  주목 인덱스 :  2
15 positive? 테스트 :  125 비교단어 :  출시 인덱스 :  15
36 positive? 테스트 :  458 비교단어 :  최고 인덱스 :  36
15 positive? 테스트 :  313 비교단어 :  출시 인덱스 :  15
30 positive? 테스트 :  520 비교단어 :  증가 인덱스 :  30
27 posit

14 positive? 테스트 :  317 비교단어 :  1위 인덱스 :  14
95 negative? 테스트 :  525 비교단어 :  우려 인덱스 :  95
59 negative? 테스트 :  198 비교단어 :  하락 인덱스 :  59
2 positive? 테스트 :  876 비교단어 :  주목 인덱스 :  2
59 negative? 테스트 :  30 비교단어 :  하락 인덱스 :  59
56 negative? 테스트 :  414 비교단어 :  리스크 인덱스 :  56
67 negative? 테스트 :  956 비교단어 :  적발 인덱스 :  67
59 negative? 테스트 :  241 비교단어 :  하락 인덱스 :  59
30 positive? 테스트 :  340 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  160 비교단어 :  하락 인덱스 :  59
2 positive? 테스트 :  92 비교단어 :  주목 인덱스 :  2
58 negative? 테스트 :  463 비교단어 :  급락 인덱스 :  58
2 positive? 테스트 :  135 비교단어 :  주목 인덱스 :  2
6 positive? 테스트 :  379 비교단어 :  수상 인덱스 :  6
8 positive? 테스트 :  692 비교단어 :  인기 인덱스 :  8
14 positive? 테스트 :  860 비교단어 :  1위 인덱스 :  14
32 positive? 테스트 :  574 비교단어 :  기부 인덱스 :  32
58 negative? 테스트 :  553 비교단어 :  급락 인덱스 :  58
15 positive? 테스트 :  982 비교단어 :  출시 인덱스 :  15
30 positive? 테스트 :  362 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  24 비교단어 :  하락 인덱스 :  59
98 negative? 테스트 :  424 비교단어 :  해지 인덱스 :  98
61 negative? 테스트 :  60

58 negative? 테스트 :  10 비교단어 :  급락 인덱스 :  58
29 positive? 테스트 :  208 비교단어 :  체결 인덱스 :  29
2 positive? 테스트 :  561 비교단어 :  주목 인덱스 :  2
59 negative? 테스트 :  35 비교단어 :  하락 인덱스 :  59
14 positive? 테스트 :  57 비교단어 :  1위 인덱스 :  14
15 positive? 테스트 :  212 비교단어 :  출시 인덱스 :  15
30 positive? 테스트 :  966 비교단어 :  증가 인덱스 :  30
30 positive? 테스트 :  811 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  27 비교단어 :  하락 인덱스 :  59
59 negative? 테스트 :  19 비교단어 :  하락 인덱스 :  59
58 negative? 테스트 :  252 비교단어 :  급락 인덱스 :  58
58 negative? 테스트 :  30 비교단어 :  급락 인덱스 :  58
5 positive? 테스트 :  361 비교단어 :  이목 인덱스 :  5
2 positive? 테스트 :  955 비교단어 :  주목 인덱스 :  2
58 negative? 테스트 :  68 비교단어 :  급락 인덱스 :  58
30 positive? 테스트 :  161 비교단어 :  증가 인덱스 :  30
13 positive? 테스트 :  235 비교단어 :  유치 인덱스 :  13
27 positive? 테스트 :  411 비교단어 :  진출 인덱스 :  27
36 positive? 테스트 :  123 비교단어 :  최고 인덱스 :  36
59 negative? 테스트 :  48 비교단어 :  하락 인덱스 :  59
59 negative? 테스트 :  948 비교단어 :  하락 인덱스 :  59
59 negative? 테스트 :  13 비교단어 :  하락 인덱스 :  59
14 positive? 테스트 :  363 비

59 negative? 테스트 :  753 비교단어 :  하락 인덱스 :  59
58 negative? 테스트 :  301 비교단어 :  급락 인덱스 :  58
59 negative? 테스트 :  149 비교단어 :  하락 인덱스 :  59
6 positive? 테스트 :  603 비교단어 :  수상 인덱스 :  6
59 negative? 테스트 :  77 비교단어 :  하락 인덱스 :  59
30 positive? 테스트 :  213 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  33 비교단어 :  하락 인덱스 :  59
27 positive? 테스트 :  844 비교단어 :  진출 인덱스 :  27
59 negative? 테스트 :  457 비교단어 :  하락 인덱스 :  59
59 negative? 테스트 :  644 비교단어 :  하락 인덱스 :  59
4 positive? 테스트 :  186 비교단어 :  돌파 인덱스 :  4
36 positive? 테스트 :  570 비교단어 :  최고 인덱스 :  36
59 negative? 테스트 :  81 비교단어 :  하락 인덱스 :  59
4 positive? 테스트 :  961 비교단어 :  돌파 인덱스 :  4
31 positive? 테스트 :  825 비교단어 :  봉사 인덱스 :  31
5 positive? 테스트 :  464 비교단어 :  이목 인덱스 :  5
5 positive? 테스트 :  502 비교단어 :  이목 인덱스 :  5
4 positive? 테스트 :  846 비교단어 :  돌파 인덱스 :  4
36 positive? 테스트 :  822 비교단어 :  최고 인덱스 :  36
47 negative? 테스트 :  287 비교단어 :  논란 인덱스 :  47
4 positive? 테스트 :  32 비교단어 :  돌파 인덱스 :  4
5 positive? 테스트 :  835 비교단어 :  이목 인덱스 :  5
59 negative? 테스트 :  891 비교단어 :

...

14 positive? 테스트 :  237 비교단어 :  1위 인덱스 :  14
2 positive? 테스트 :  35 비교단어 :  주목 인덱스 :  2
37 positive? 테스트 :  312 비교단어 :  새로운 인덱스 :  37
30 positive? 테스트 :  998 비교단어 :  증가 인덱스 :  30
2 positive? 테스트 :  52 비교단어 :  주목 인덱스 :  2
27 positive? 테스트 :  724 비교단어 :  진출 인덱스 :  27
58 negative? 테스트 :  24 비교단어 :  급락 인덱스 :  58
36 positive? 테스트 :  973 비교단어 :  최고 인덱스 :  36
79 negative? 테스트 :  752 비교단어 :  문제 인덱스 :  79
30 positive? 테스트 :  438 비교단어 :  증가 인덱스 :  30
58 negative? 테스트 :  512 비교단어 :  급락 인덱스 :  58
36 positive? 테스트 :  447 비교단어 :  최고 인덱스 :  36
14 positive? 테스트 :  173 비교단어 :  1위 인덱스 :  14
2 positive? 테스트 :  528 비교단어 :  주목 인덱스 :  2
13 positive? 테스트 :  29 비교단어 :  유치 인덱스 :  13
4 positive? 테스트 :  21 비교단어 :  돌파 인덱스 :  4
14 positive? 테스트 :  128 비교단어 :  1위 인덱스 :  14
14 positive? 테스트 :  617 비교단어 :  1위 인덱스 :  14
95 negative? 테스트 :  61 비교단어 :  우려 인덱스 :  95
13 positive? 테스트 :  104 비교단어 :  유치 인덱스 :  13
14 positive? 테스트 :  761 비교단어 :  1위 인덱스 :  14
59 negative? 테스트 :  398 비교단어 :  하락 인덱스 :  59
14 positive? 테스트 :  668

14 positive? 테스트 :  60 비교단어 :  1위 인덱스 :  14
10 positive? 테스트 :  528 비교단어 :  진화 인덱스 :  10
30 positive? 테스트 :  18 비교단어 :  증가 인덱스 :  30
14 positive? 테스트 :  403 비교단어 :  1위 인덱스 :  14
27 positive? 테스트 :  529 비교단어 :  진출 인덱스 :  27
30 positive? 테스트 :  666 비교단어 :  증가 인덱스 :  30
80 negative? 테스트 :  274 비교단어 :  직격탄 인덱스 :  80
17 positive? 테스트 :  934 비교단어 :  오픈 인덱스 :  17
3 positive? 테스트 :  553 비교단어 :  호응 인덱스 :  3
2 positive? 테스트 :  553 비교단어 :  주목 인덱스 :  2
28 positive? 테스트 :  604 비교단어 :  부상 인덱스 :  28
14 positive? 테스트 :  145 비교단어 :  1위 인덱스 :  14
15 positive? 테스트 :  663 비교단어 :  출시 인덱스 :  15
30 positive? 테스트 :  45 비교단어 :  증가 인덱스 :  30
58 negative? 테스트 :  85 비교단어 :  급락 인덱스 :  58
10 positive? 테스트 :  489 비교단어 :  진화 인덱스 :  10
30 positive? 테스트 :  167 비교단어 :  증가 인덱스 :  30
29 positive? 테스트 :  69 비교단어 :  체결 인덱스 :  29
30 positive? 테스트 :  979 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  156 비교단어 :  하락 인덱스 :  59
2 positive? 테스트 :  137 비교단어 :  주목 인덱스 :  2
30 positive? 테스트 :  173 비교단어 :  증가 인덱스 :  30
2 positive? 테스트 :  7

92 negative? 테스트 :  57 비교단어 :  울상 인덱스 :  92
3 positive? 테스트 :  768 비교단어 :  호응 인덱스 :  3
24 positive? 테스트 :  367 비교단어 :  대세 인덱스 :  24
47 negative? 테스트 :  789 비교단어 :  논란 인덱스 :  47
2 positive? 테스트 :  569 비교단어 :  주목 인덱스 :  2
14 positive? 테스트 :  585 비교단어 :  1위 인덱스 :  14
58 negative? 테스트 :  510 비교단어 :  급락 인덱스 :  58
25 positive? 테스트 :  878 비교단어 :  트렌드 인덱스 :  25
2 positive? 테스트 :  100 비교단어 :  주목 인덱스 :  2
2 positive? 테스트 :  817 비교단어 :  주목 인덱스 :  2
4 positive? 테스트 :  72 비교단어 :  돌파 인덱스 :  4
30 positive? 테스트 :  490 비교단어 :  증가 인덱스 :  30
14 positive? 테스트 :  508 비교단어 :  1위 인덱스 :  14
2 positive? 테스트 :  892 비교단어 :  주목 인덱스 :  2
9 positive? 테스트 :  448 비교단어 :  열풍 인덱스 :  9
11 positive? 테스트 :  306 비교단어 :  대박 인덱스 :  11
2 positive? 테스트 :  937 비교단어 :  주목 인덱스 :  2
2 positive? 테스트 :  158 비교단어 :  주목 인덱스 :  2
2 positive? 테스트 :  695 비교단어 :  주목 인덱스 :  2
2 positive? 테스트 :  335 비교단어 :  주목 인덱스 :  2
4 positive? 테스트 :  105 비교단어 :  돌파 인덱스 :  4
4 positive? 테스트 :  72 비교단어 :  돌파 인덱스 :  4
4 positive? 테스트 :  126 비교단어 :  돌파 인덱스 

2 positive? 테스트 :  735 비교단어 :  주목 인덱스 :  2
4 positive? 테스트 :  66 비교단어 :  돌파 인덱스 :  4
30 positive? 테스트 :  318 비교단어 :  증가 인덱스 :  30
27 positive? 테스트 :  65 비교단어 :  진출 인덱스 :  27
30 positive? 테스트 :  403 비교단어 :  증가 인덱스 :  30
30 positive? 테스트 :  678 비교단어 :  증가 인덱스 :  30
29 positive? 테스트 :  137 비교단어 :  체결 인덱스 :  29
2 positive? 테스트 :  327 비교단어 :  주목 인덱스 :  2
2 positive? 테스트 :  777 비교단어 :  주목 인덱스 :  2
56 negative? 테스트 :  507 비교단어 :  리스크 인덱스 :  56
30 positive? 테스트 :  279 비교단어 :  증가 인덱스 :  30
2 positive? 테스트 :  883 비교단어 :  주목 인덱스 :  2
28 positive? 테스트 :  990 비교단어 :  부상 인덱스 :  28
27 positive? 테스트 :  721 비교단어 :  진출 인덱스 :  27
30 positive? 테스트 :  362 비교단어 :  증가 인덱스 :  30
28 positive? 테스트 :  479 비교단어 :  부상 인덱스 :  28
6 positive? 테스트 :  992 비교단어 :  수상 인덱스 :  6
2 positive? 테스트 :  448 비교단어 :  주목 인덱스 :  2
27 positive? 테스트 :  772 비교단어 :  진출 인덱스 :  27
4 positive? 테스트 :  500 비교단어 :  돌파 인덱스 :  4
14 positive? 테스트 :  120 비교단어 :  1위 인덱스 :  14
4 positive? 테스트 :  8 비교단어 :  돌파 인덱스 :  4
36 positive? 테스트 :  387 비교단어 : 

14 positive? 테스트 :  263 비교단어 :  1위 인덱스 :  14
30 positive? 테스트 :  302 비교단어 :  증가 인덱스 :  30
96 negative? 테스트 :  176 비교단어 :  중단 인덱스 :  96
13 positive? 테스트 :  503 비교단어 :  유치 인덱스 :  13
5 positive? 테스트 :  843 비교단어 :  이목 인덱스 :  5
52 negative? 테스트 :  516 비교단어 :  피해 인덱스 :  52
59 negative? 테스트 :  121 비교단어 :  하락 인덱스 :  59
30 positive? 테스트 :  821 비교단어 :  증가 인덱스 :  30
58 negative? 테스트 :  24 비교단어 :  급락 인덱스 :  58
30 positive? 테스트 :  18 비교단어 :  증가 인덱스 :  30
34 positive? 테스트 :  156 비교단어 :  신제품 인덱스 :  34
58 negative? 테스트 :  53 비교단어 :  급락 인덱스 :  58
37 positive? 테스트 :  793 비교단어 :  새로운 인덱스 :  37
58 negative? 테스트 :  51 비교단어 :  급락 인덱스 :  58
58 negative? 테스트 :  31 비교단어 :  급락 인덱스 :  58
95 negative? 테스트 :  219 비교단어 :  우려 인덱스 :  95
30 positive? 테스트 :  521 비교단어 :  증가 인덱스 :  30
59 negative? 테스트 :  845 비교단어 :  하락 인덱스 :  59
95 negative? 테스트 :  508 비교단어 :  우려 인덱스 :  95
37 positive? 테스트 :  426 비교단어 :  새로운 인덱스 :  37
5 positive? 테스트 :  494 비교단어 :  이목 인덱스 :  5
58 negative? 테스트 :  133 비교단어 :  급락 인덱스 :  58
14 positive? 테스트

In [14]:
my_df

,text,label
0,2020년에는 상대평가제도도 폐지소통강화 위해 호칭도 TL로 통일 아시아경제 안하늘...,1
1,아시아경제 오주연 기자 오션브릿지오션브릿지241790코스닥증권정보현재가15150전일...,1
2,인사제도 파격적 개편최태원 회장의 딥체인지 실현 아시아경제 안하늘 기자 SK하이닉스...,0
3,아시아경제 송화정 기자신한금융투자는 26일 SK하이닉스SK하이닉스000660코스피증...,1
4,아시아경제 오주연 기자 하나금융투자가 27일 SK하이닉스SK하이닉스000660코스피...,0
...,...,...
9504,美 반도체기업에 도넘은 요구 미국 상무부가 삼성전자를 비롯한 글로벌 반도체 기업에...,0
9505,지나 러몬도 미국 상무장관 사진 출처 연합뉴스 미국 백악관과 상무부가 삼성전자 T...,-1
9506,사진은 해당 기사와는 무관함 매경 DB 삼성전자 현대자동차 SK하이닉스 LG전자 포...,1
9507,미국 정부가 삼성전자 TSMC 인텔 등 반도체 기업들에 45일 내에 재고·주문·판매...,-1


In [15]:
my_df.label.value_counts()

 1    5984
-1    2426
 0    1099
Name: label, dtype: int64

In [16]:
my_df.to_csv('./data/labeling.csv', index=False)

In [17]:
my_df.text[0]

'2020년에는 상대평가제도도 폐지소통강화 위해 호칭도 TL로 통일 아시아경제 안하늘 기자 SK하이닉스가 기술력이 높은 엔지니어의 경우 정년60세에 관계 없이 일할 수 있는 새로운 제도를 도입한다 전문성을 갖춘 우수한 엔지니어들은 정년 이후에도 활발하게 연구개발·제조·분석 등의 업무에 매진할 수 있게 된다SK하이닉스는 27일 이천캠퍼스에서 이석희 사장CEO과 임직원 400여명이 참석한 가운데 왁자지껄 콘서트’를 열고 엔지니어 정년 이후 근무 가능한 제도 도입 협업 강화를 위한 평가제도 개선 등을 골자로 하는 CEO 공감경영 선언을 발표했다 이날 선언은 해외 혁신기업 문화 체험을 위해 모집된 직원들이 지난 9월 현장을 방문한 후 내놓은 목소리를 바탕으로 구성됐다먼저 SK하이닉스는 우수 엔지니어가 정년이 지나서도 전문성에 따라 계속 근무할 수 있는 제도를 도입한다 SK하이닉스는 이 제도 도입으로 오랫동안 회사 성장에 기여한 우수한 기술인력들이 정년을 넘어서도 회사에서 실력을 발휘할 수 있게 되어 개개인은 물론 회사의 기술역량 또한 한층 높일 수 있을 것으로 기대하고 있다 유만석 SK하이닉스 HR담당 전무는 반도체 개발·제조 분야의 숙련된 인력에 대한 수요는 지속적으로 늘고 있다”며 새롭게 도입하는 본 제도는 내년 정년 대상자부터 시행할 예정”이라고 밝혔다또 협업을 통해 경쟁력을 강화할 수 있도록 동료간 불필요한 경쟁을 부추길 수 있는 상대평가 제도를 2020년부터 폐지한다 연초에 목표를 세우고 반기와 연말에 평가를 받던 정기평가는 급격한 경영환경 변화에 대응하기 위해 프로젝트별 상시 업무평가로 대체된다 이를 통해 팀원은 리더와 업무 수행과정 중 지속적으로 소통하며 다양한 성과를 적기에 인정받을 수 있게 될 것으로 기대된다SK하이닉스는 이날 발표한 공감경영 선언에 대한 임직원들의 다양한 의견을 수렴해 빠른 시간 내에 세부적인 기준과 시행 시기를 확정한다는 방침이다이밖에 SK하이닉스는 세대·직위·직군간 소통을 강화하고 직원들의 자발적 의견 개진을 활성화하기 위해 20

In [ ]:
하이닉스 --> 하이닉스 좋다 --> 기사 긍정 --> 기사의 분류결과 <- 을 앙상블을하는 거
하이닉스 --> 긍정판단 --> 주식이 올르거라고 판단한 거라 볼수 있다 <- 기사들마다 정확도를 낼때 쓸수있는 변수


하이닉스 --> 감성사전 구축 (레이블 : 주식등락) --> 주식등락이랑 비교를 하면서 정확도를 본것


 보 팅(LSTM,머신러닝) + 기사긍부정 + 강화학습 --> 분류 